<a href="https://colab.research.google.com/github/Eshita781996/SCD/blob/main/SCD3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
!pip -q install findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages io.delta:delta-core_2.12:0.7.0 --conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog pyspark-shell'

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName('delta_session').getOrCreate()

In [ ]:
# employees_df = spark.createDataFrame([(1, 'John', 'Sydney', 35000.00),(2, 'Peter', 'Melbourne', 45000.00),(3, 'Sam', 'Sydney',55000.00)],
#                                      schema='emp_id int,emp_name string,emp_city string,emp_salary float')

employees_delta_df = spark.createDataFrame([(1, 'Max', 'Melbourne', 55002.00),(4, 'Bella', 'Brisbane', 48002.00),(3, 'Yen', 'Melbourne', 55002.00)],
                                           schema='emp_id int,emp_name string,emp_city string,emp_salary float')

In [ ]:
employees_delta_df.createOrReplaceTempView('INPUT_VIEW')

In [ ]:
employees_delta_df.show()

+------+--------+---------+----------+
|emp_id|emp_name| emp_city|emp_salary|
+------+--------+---------+----------+
|     1|     Max|Melbourne|   55002.0|
|     4|   Bella| Brisbane|   48002.0|
|     3|     Yen|Melbourne|   55002.0|
+------+--------+---------+----------+



In [ ]:
#For the first time
from delta.tables import *
if not DeltaTable.isDeltaTable(spark,'employees'):
  employees_delta_df = employees_delta_df.withColumn('old_emp_salary',employees_delta_df['emp_salary'])
  employees_delta_df = employees_delta_df.withColumn('old_emp_city',employees_delta_df['emp_city'])
  employees_delta_df.write.format("delta").save("employees")
else:
  df = spark.read.format("delta").load("employees")
  df.createOrReplaceTempView('employees')
  DF_STAGED_UPDATE = spark.sql(f"merge into employees using INPUT_VIEW on employees.emp_id=INPUT_VIEW.emp_id\
   when matched and (employees.emp_salary<>INPUT_VIEW.emp_salary ) and employees.emp_id=INPUT_VIEW.emp_id \
  then update set employees.emp_salary=INPUT_VIEW.emp_salary   when not matched then insert (emp_id,emp_name,emp_city,emp_salary,old_emp_city,old_emp_salary) \
  VALUES (INPUT_VIEW.emp_id,INPUT_VIEW.emp_name,INPUT_VIEW.emp_city,INPUT_VIEW.emp_salary,INPUT_VIEW.emp_city,INPUT_VIEW.emp_salary) ")
  df = spark.sql('select * from employees')
  df.show()
  df.write.format("delta").mode("overwrite").save("employees")


+------+--------+---------+----------+------------+--------------+
|emp_id|emp_name| emp_city|emp_salary|old_emp_city|old_emp_salary|
+------+--------+---------+----------+------------+--------------+
|     2|  Jessie| Brisbane|   48001.0|    Brisbane|       48001.0|
|     4|   Bella| Brisbane|   48002.0|    Brisbane|       48001.0|
|     3|     Yen|Melbourne|   55002.0|   Melbourne|       55002.0|
|     1|     Max|Melbourne|   55002.0|   Melbourne|       55001.0|
+------+--------+---------+----------+------------+--------------+



In [ ]:
spark.read.format("delta").load("employees").show()

+------+--------+---------+----------+------------+--------------+
|emp_id|emp_name| emp_city|emp_salary|old_emp_city|old_emp_salary|
+------+--------+---------+----------+------------+--------------+
|     2|  Jessie| Brisbane|   48001.0|    Brisbane|       48001.0|
|     4|   Bella| Brisbane|   48002.0|    Brisbane|       48001.0|
|     3|     Yen|Melbourne|   55002.0|   Melbourne|       55002.0|
|     1|     Max|Melbourne|   55002.0|   Melbourne|       55001.0|
+------+--------+---------+----------+------------+--------------+

